In [1]:
!make -C csrc/bcc/libbpf-tools

make: Entering directory '/Users/szhong/Downloads/bpf-study/csrc/bcc/libbpf-tools'
make: Nothing to be done for 'all'.
make: Leaving directory '/Users/szhong/Downloads/bpf-study/csrc/bcc/libbpf-tools'


In [2]:
!make -C csrc/bpftool/src bpftool

make: Entering directory '/Users/szhong/Downloads/bpf-study/csrc/bpftool/src'
...                        libbfd: [ OFF ]
...               clang-bpf-co-re: [ on  ]
...                          llvm: [ on  ]
...                        libcap: [ OFF ]
make: 'bpftool' is up to date.
make: Leaving directory '/Users/szhong/Downloads/bpf-study/csrc/bpftool/src'


In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
import pickle

from depsurf import BPFObject, VersionGroup, gen_report
from utils import BCC_OBJ_PATH, OUTPUT_PATH

results = {}
for path in sorted(BCC_OBJ_PATH.glob("*.bpf.o")):
    bpf = BPFObject(path)
    result_path = OUTPUT_PATH / "bcc" / f"{bpf.name}.log"
    with open(result_path, "w") as f:
        results[bpf.name] = gen_report(
            bpf.deps, [VersionGroup.REGULAR, VersionGroup.ARCH], f
        )
    print(f"Report for {bpf.name} saved to {result_path}")


with open(OUTPUT_PATH / "bcc.pkl", "wb") as f:
    pickle.dump(results, f)

[       groups.py:68 ] INFO: Loading funcs from /Users/szhong/Downloads/bpf-study/data/dwarf_funcs/4.4.0-21-generic-amd64.jsonl


Report for bashreadline saved to /Users/szhong/Downloads/bpf-study/output/bcc/bashreadline.log


[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.4.0-21-generic-amd64.jsonl
[       groups.py:68 ] INFO: Loading funcs from /Users/szhong/Downloads/bpf-study/data/dwarf_funcs/4.8.0-22-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.8.0-22-generic-amd64.jsonl
[       groups.py:68 ] INFO: Loading funcs from /Users/szhong/Downloads/bpf-study/data/dwarf_funcs/4.10.0-19-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.10.0-19-generic-amd64.jsonl
[       groups.py:68 ] INFO: Loading funcs from /Users/szhong/Downloads/bpf-study/data/dwarf_funcs/4.13.0-16-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.13.0-16-generic-amd64.jsonl
[       groups.py:68 ] INFO: Loading funcs from /Users/szhong/Downloads/bpf-study/data/dwarf_funcs/4.15.0-20-generic

Report for bindsnoop saved to /Users/szhong/Downloads/bpf-study/output/bcc/bindsnoop.log
Report for biolatency saved to /Users/szhong/Downloads/bpf-study/output/bcc/biolatency.log
Report for biopattern saved to /Users/szhong/Downloads/bpf-study/output/bcc/biopattern.log
Report for biosnoop saved to /Users/szhong/Downloads/bpf-study/output/bcc/biosnoop.log
Report for biostacks saved to /Users/szhong/Downloads/bpf-study/output/bcc/biostacks.log
Report for biotop saved to /Users/szhong/Downloads/bpf-study/output/bcc/biotop.log
Report for bitesize saved to /Users/szhong/Downloads/bpf-study/output/bcc/bitesize.log
Report for cachestat saved to /Users/szhong/Downloads/bpf-study/output/bcc/cachestat.log
Report for capable saved to /Users/szhong/Downloads/bpf-study/output/bcc/capable.log
Report for cpudist saved to /Users/szhong/Downloads/bpf-study/output/bcc/cpudist.log
Report for cpufreq saved to /Users/szhong/Downloads/bpf-study/output/bcc/cpufreq.log
Report for drsnoop saved to /Users/szho

[     syscalls.py:54 ] WARNING: Unknown syscall at 513: ffffffff81a011c8 -> ffffffff8102ddd0
[     syscalls.py:54 ] WARNING: Unknown syscall at 513: ffffffff81a01208 -> ffffffff8102dc30
[     syscalls.py:54 ] WARNING: Unknown syscall at 513: ffffffff81a011e8 -> ffffffff8102beb0
[     syscalls.py:54 ] WARNING: Unknown syscall at 513: ffffffff81e011a8 -> ffffffff8102cbd0
[     syscalls.py:54 ] WARNING: Unknown syscall at 513: ffffffff81e011c8 -> ffffffff8102de00
[     syscalls.py:54 ] WARNING: Unknown syscall at 513: ffffffff820011c8 -> ffffffff8102f040
[     syscalls.py:54 ] WARNING: Unknown syscall at 513: ffffffff82001288 -> ffffffff81030e10
[     syscalls.py:27 ] WARNING: sys_call_table size is 0. Using hardcoded size
[     syscalls.py:27 ] WARNING: sys_call_table size is 0. Using hardcoded size


Report for execsnoop saved to /Users/szhong/Downloads/bpf-study/output/bcc/execsnoop.log
Report for exitsnoop saved to /Users/szhong/Downloads/bpf-study/output/bcc/exitsnoop.log
Report for filelife saved to /Users/szhong/Downloads/bpf-study/output/bcc/filelife.log
Report for filetop saved to /Users/szhong/Downloads/bpf-study/output/bcc/filetop.log
Report for fsdist saved to /Users/szhong/Downloads/bpf-study/output/bcc/fsdist.log
Report for fsslower saved to /Users/szhong/Downloads/bpf-study/output/bcc/fsslower.log
Report for funclatency saved to /Users/szhong/Downloads/bpf-study/output/bcc/funclatency.log
Report for futexctn saved to /Users/szhong/Downloads/bpf-study/output/bcc/futexctn.log
Report for gethostlatency saved to /Users/szhong/Downloads/bpf-study/output/bcc/gethostlatency.log
Report for hardirqs saved to /Users/szhong/Downloads/bpf-study/output/bcc/hardirqs.log
Report for javagc saved to /Users/szhong/Downloads/bpf-study/output/bcc/javagc.log
Report for klockstat saved to /